In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zhuhai,CN,22.2769,113.5678,69.71,63,20,9.22,few clouds
1,1,La Ronge,CA,55.1001,-105.2842,-3.75,71,40,2.30,scattered clouds
2,2,Port Blair,IN,11.6667,92.7500,82.44,65,40,10.36,scattered clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,38.86,81,75,6.91,broken clouds
4,4,Dingle,PH,10.9995,122.6711,86.76,59,97,11.03,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 70
What is the maximun temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Port Blair,IN,11.6667,92.7500,82.44,65,40,10.36,scattered clouds
4,4,Dingle,PH,10.9995,122.6711,86.76,59,97,11.03,overcast clouds
6,6,Naifaru,MV,5.4444,73.3657,81.18,72,95,2.21,overcast clouds
17,17,Clarence Town,AU,-32.5833,151.7833,80.82,53,94,10.29,overcast clouds
32,32,Waingapu,ID,-9.6567,120.2641,85.59,67,40,8.57,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

156


City_ID                156
City                   156
Country                156
Lat                    156
Lng                    156
Max Temp               156
Humidity               156
Cloudiness             156
Wind Speed             156
Current Description    156
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

156


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Port Blair,IN,11.6667,92.7500,82.44,65,40,10.36,scattered clouds
4,4,Dingle,PH,10.9995,122.6711,86.76,59,97,11.03,overcast clouds
6,6,Naifaru,MV,5.4444,73.3657,81.18,72,95,2.21,overcast clouds
17,17,Clarence Town,AU,-32.5833,151.7833,80.82,53,94,10.29,overcast clouds
32,32,Waingapu,ID,-9.6567,120.2641,85.59,67,40,8.57,scattered clouds


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Blair,IN,82.44,scattered clouds,11.6667,92.7500,
4,Dingle,PH,86.76,overcast clouds,10.9995,122.6711,
6,Naifaru,MV,81.18,overcast clouds,5.4444,73.3657,
17,Clarence Town,AU,80.82,overcast clouds,-32.5833,151.7833,
32,Waingapu,ID,85.59,scattered clouds,-9.6567,120.2641,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Blair,IN,82.44,scattered clouds,11.6667,92.7500,
4,Dingle,PH,86.76,overcast clouds,10.9995,122.6711,
6,Naifaru,MV,81.18,overcast clouds,5.4444,73.3657,
17,Clarence Town,AU,80.82,overcast clouds,-32.5833,151.7833,
32,Waingapu,ID,85.59,scattered clouds,-9.6567,120.2641,


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))